### Investigate the relationship between option order book update frequencies and the future prices update frequencies
* data processing
    * find the ATM call and put, most updated call and most updated put
    * for each single instrument in a day
    * filter out snapshot
    * remove duplicated timepoint
    * round to nearest minutes
    * groupby timestamp and count -> update frequency/min

In [ ]:
from croqr.common.config import LOCAL_DATA_DIR
import os
import pandas as pd
from datetime import datetime, timedelta
import croqr.data.deribit.data_utils as datautils
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import time
from croqr.common.utils import print_time_from_t0
from croqr.common.dt_utils import DataTimeUtils
from importlib import reload
import matplotlib.pyplot as plt

In [ ]:
reload(datautils)

In [ ]:
TARDIS_DATA_PATH = os.path.join(LOCAL_DATA_DIR, 'tardis') 

In [ ]:
os.listdir(TARDIS_DATA_PATH)

### load and process options data

#### incremental_book_L2

In [ ]:
symbol = 'OPTIONS'
date = "2021-10-01"
data_type = "incremental_book_L2" #"quotes"

In [ ]:
t0 = time.time()
fname = 'deribit_{}_{}_{}.csv.gz'.format(data_type, date, symbol)
df = pd.read_csv(os.path.join(TARDIS_DATA_PATH, fname))
print_time_from_t0(t0)

In [ ]:
df.head()

In [ ]:
df1 = datautils.DeribitUtils.parse_symbol_col(df)

In [ ]:
df1.head()

In [ ]:
df1['timestamp_dt'] = pd.to_datetime(df1['timestamp'], unit='us')

In [ ]:
df1['local_dt'] = pd.to_datetime(df1['local_timestamp'], unit='us')

In [ ]:
df1

In [ ]:
all_expire_dates = sorted(list(df1['expire'].unique()))
all_expire_dates

In [ ]:
df1a = df1[df1['expire']==all_expire_dates[0]].query('instrument=="BTC"').query('type=="C"').query('strike==44000')

In [ ]:
df1a['is_snapshot'].value_counts()

### Load and process futures data

In [ ]:
df1a['timestamp_dt'].nunique()

In [ ]:
df1a.shape

In [ ]:
df1a.query('is_snapshot==True').sort_values('price', ascending=False)

In [ ]:
df1a.query('is_snapshot==False')['timestamp_dt'].nunique()

In [ ]:
df1a.tail()

In [ ]:
df1a['timestamp_dt'].dt.round('T')

In [ ]:
df1b = df1a.query('is_snapshot==False')

In [ ]:
df1b['expire'] = df1b['expire'] + timedelta(hours=8)

In [ ]:
df1b['t2m'] = (df1b['expire'] - df1b['timestamp_dt'].dt.floor('H')).dt.total_seconds()/3600

In [ ]:
(df1b.groupby('t2m')['timestamp_dt'].nunique()/60).plot()

In [ ]:
df1b['dt_1m'] = df1b['timestamp_dt'].dt.ceil('T')

In [ ]:
df1b.groupby('dt_1m')['timestamp_dt'].nunique().plot()

#### full order book update can be too noisy, check quote data

In [ ]:
symbol = 'OPTIONS'
date = "2021-10-01"
data_type = "quotes" #"quotes"

In [ ]:
t0 = time.time()
fname = 'deribit_{}_{}_{}.csv.gz'.format(data_type, date, symbol)
df = pd.read_csv(os.path.join(TARDIS_DATA_PATH, fname))
print_time_from_t0(t0)

In [ ]:
df1 = df.pipe(datautils.DeribitUtils.parse_symbol_col).pipe(datautils.DeribitUtils.parse_time_col).query('t2m<=168 and t2m>0')

In [ ]:
df1.head(2)

In [ ]:
df2_btc_c = df1.query('instrument=="BTC" and type=="C"')

In [ ]:
for strike in sorted(list(df2_btc_c['strike'].unique())):
    #strike=44000
    print('strike=', strike)
    df2plot = df2_btc_c.query('strike==@strike').query('t2m<8')
    prices = sorted(list(df2plot['ask_price'].unique()))
    print('ask prices', prices)
    for price in prices:
        df2plot.query('ask_price==@price').groupby('timestamp_dt')['ask_amount'].max().plot()
    if len(prices)<13:
        plt.legend(prices)
    plt.show()

#### check quote data for futures

In [ ]:
symbol = 'BTC-PERPETUAL'
date = "2021-10-01"
data_type = "quotes" #"quotes"

In [ ]:
t0 = time.time()
fname = 'deribit_{}_{}_{}.csv.gz'.format(data_type, date, symbol)
df = pd.read_csv(os.path.join(TARDIS_DATA_PATH, fname))
print_time_from_t0(t0)

In [ ]:
df1 = df.pipe(datautils.DeribitUtils.parse_futureSymbol_col).pipe(datautils.DeribitUtils.parse_time_col)

df1_btc = df1.query('instrument=="BTC"')

df1_btc['update'] = (df1_btc['ask_price'].diff()!=0) | (df1_btc['bid_price'].diff()!=0)

df1_btc['timestamp_dt_1h'] = df1_btc['timestamp_dt'].dt.ceil('H')

(df1_btc.groupby('timestamp_dt_1h')['update'].sum()/60).plot()

In [ ]:
symbol = 'BTC-PERPETUAL'
date = "2021-10-01"
data_type = "quotes" #"quotes"

In [ ]:
t0 = time.time()
fname = 'deribit_{}_{}_{}.csv.gz'.format(data_type, date, symbol)
df = pd.read_csv(os.path.join(TARDIS_DATA_PATH, fname))
print_time_from_t0(t0)

In [ ]:
## analyze a list of dates for BTC PERP

In [ ]:
symbol = 'BTC-PERPETUAL'
data_type = "quotes" #"quotes"
dates1 = pd.date_range(datetime(2021,1,1), datetime(2021,1,7))
dates2 = pd.date_range(datetime(2021,5,15), datetime(2021,5,21))


In [ ]:
for date in dates1:
    date_str = date.strftime('%Y-%m-%d')
    fname = 'deribit_{}_{}_{}.csv.gz'.format(data_type, date_str, symbol)
    df = pd.read_csv(os.path.join(TARDIS_DATA_PATH, fname))
    

In [ ]:
df_combined2 = pd.concat([pd.read_csv(os.path.join(TARDIS_DATA_PATH,  'deribit_{}_{}_{}.csv.gz'.format(data_type, date.strftime('%Y-%m-%d'), symbol))) for date in dates2])

In [ ]:
df_combined1 = pd.concat([pd.read_csv(os.path.join(TARDIS_DATA_PATH,  'deribit_{}_{}_{}.csv.gz'.format(data_type, date.strftime('%Y-%m-%d'), symbol))) for date in dates1])

In [ ]:
df1 = df_combined1.pipe(datautils.DeribitUtils.parse_futureSymbol_col).pipe(datautils.DeribitUtils.parse_time_col)

df1_btc = df1.query('instrument=="BTC"')

df1_btc['update'] = (df1_btc['ask_price'].diff()!=0) | (df1_btc['bid_price'].diff()!=0)

df1_btc['timestamp_dt_1h'] = df1_btc['timestamp_dt'].dt.ceil('H')

(df1_btc.groupby('timestamp_dt_1h')['update'].sum()/60).plot()